In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import bs4
import re
import copy
import collections as co

import plotly as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode
import matplotlib.pyplot as mplt



from wordcloud import WordCloud
from wordcloud import STOPWORDS
from PIL import Image
import requests
from io import BytesIO

On importe AS

In [297]:
#Cellule léo
#AS= pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\All_Streaming_Shows.csv',sep=',')
#AS.shape[0] #12 353
#AS.head(5)

In [2]:
#Cellule Myrtille
AS= pd.read_csv(r'C:\Users\Utilisateur\Documents\GitHub\Projet_Python_2A\All_Streaming_Shows.csv',sep=',')
AS.shape[0] #12 353
AS.head(5)

,Series Title,Year Released,Content Rating,IMDB Rating,R Rating,Genre,Description,No of Seasons,Streaming Platform
0,Breaking Bad,2008,18+,9.5,100,"Crime,Drama","When Walter White, a New Mexico chemistry teac...",5Seasons,Netflix
1,Game of Thrones,2011,18+,9.3,99,"Action & Adventure,Drama",Seven noble families fight for control of the ...,8Seasons,"HBO MAX,HBO"
2,Rick and Morty,2013,18+,9.2,97,"Animation,Comedy",Rick is a mentally-unbalanced but scientifical...,4Seasons,"Free Services,HBO MAX,Hulu"
3,Stranger Things,2016,16+,8.8,96,"Drama,Fantasy","When a young boy vanishes, a small town uncove...",3Seasons,Netflix
4,The Boys,2019,18+,8.7,95,"Action & Adventure,Comedy",A group of vigilantes known informally as “The...,2Seasons,Prime Video


On créé une variable Top IMDB qui vaut 1 si la série fait partie du top 250 d'IMDB et 0 sinon.
On transforme les variables No of Seasons et Top IMDB en entier pour pouvoir faire des opérations dessus.

In [3]:
AS['Top IMDB'] = AS['IMDB Rating'].apply(lambda x: '1' if x >= 8.4 else '0')

regex = r'([0-9]?[0-9])'
AS['No of Seasons'] = AS['No of Seasons'].str.extract(regex,expand=True)
AS['No of Seasons'] = AS['No of Seasons'].fillna(0)
AS['No of Seasons'] = AS['No of Seasons'].astype(str).astype(int)
AS['Top IMDB'] = AS['Top IMDB'].astype(str).astype(int)

In [300]:
#AS.dtypes

### Création du AS_N ac seulement Netflix

On a 1427 titres.

In [301]:
AS['Streaming Platform'].isna().sum() #1983 valeurs manquantes
AS['Streaming Platform'].isin(['Netflix']).sum() #1 427 titres Netflix

1427

In [8]:
#gfg_csv_data = AS.to_csv('AS.csv', index = True) 

In [9]:
#AS_N=AS[AS['Streaming Platform'].isin(['Netflix'])]
#gfg_csv_data = AS_N.to_csv('AS_N.csv', index = True) 

Sur les 1427 titres, 1321 ont une note IMdB renseignée

In [7]:
AS_N['IMDB Rating'].notna().sum() #1341 notes IMdB
AS_N.head(5)

NameError: name 'AS_N' is not defined

# STATISTIQUES DESCRIPTIVES

Baisse en 2020 -> année tronquée + corona

## Années de création des séries sur Netflix

In [305]:
#creating the plot of number of movies released each year and the cumsum of that calculation
moviesReleasedEachYear = AS_N['Year Released'].value_counts().sort_index()
cumsumMoviesReleased = AS_N['Year Released'].value_counts().sort_index().cumsum()

trace1 = go.Bar(x = list(moviesReleasedEachYear.index), y = list(moviesReleasedEachYear.values),name = 'Movies Released Each Year')
trace2 = go.Scatter(x = list(cumsumMoviesReleased.index), 
                    y = list(cumsumMoviesReleased.values), 
                    mode='lines',name = 'Trendline of Number of Movies')

fig = plt.subplots.make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(trace1, secondary_y=False)
fig.add_trace(trace2, secondary_y=True)

fig.update_layout(template = 'plotly_white',margin=dict(l=80, r=80, t=25, b=10),
                  title = { 'text' : '<b>Séries crées dans le Temps</b>', 'x' : 0.5},
                 font_family = 'Calibri',title_font_color= 'black', showlegend = True)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01))

fig.show()

## Travail sur le genres

### Contenu ajouté sur Netflix en fonction du genre

In [279]:
"""-------------------------Fonction qui récupère les genres des Séries-------------------------"""


def recup_cat(data):
    
    col = "Genre"
    categories = ", ".join(data['Genre']).split(", ")
    #print (categories)
    categories = ",".join(data['Genre']).split(",")
    #print (categories)
    
    """on enlève les catégories débiles"""
    for i in ['2018','2017','2016','2019','2020','Netflix','2015','2012','2007','2004',
              '2010','2013','2009','1969','2005','2006','2014','1999','1972','2002','2011',
              '2008','Hulu','Prime Video','fuboTV']:
        while i in categories:
            categories.remove(i)
    return(categories)

In [280]:
"""-------------------------Fonction qui retourne les effectifs de chaque genre-------------------------"""

def count_genra (data_f,nombre=7):
    
    categories=recup_cat(data_f)

    #print(categories)

    """On compte le nombre d'occurences de chaque type et on garde les 50 qui apparaissent le plus souvent"""
    counter_list = co.Counter(categories).most_common(nombre)
    #print ((counter_list))

    """On conserve le premier élément de chaque couple de counter_list (ie. sur le genre) et on inverse l'ordre de la liste"""
    labels = [_[0] for _ in counter_list][::-1]
    #print (labels)

    """Idem pour le nombre d'occurrences"""
    values = [_[1] for _ in counter_list][::-1]
    return (values,labels)

In [281]:
count_genra(AS_N,9)

([70, 102, 104, 190, 206, 211, 216, 356, 575],
 ['Family',
  'Romance',
  'Reality',
  'Animation',
  'Crime',
  'Action & Adventure',
  'Documentary',
  'Comedy',
  'Drama'])

In [282]:
"""-------------------------Fonction qui trace un graph des effectifs des genres-------------------------"""

def trace(values,labels,nombre=7,date="Netflix") :  
    colors =['#DF0101','#086A87'] *int(nombre/2) if (nombre%2 == 0) else ['#086A87']+(['#DF0101','#086A87']*int(nombre/2))
    trace1 = go.Bar(y=labels, x=values, orientation="h",marker_color=colors, name="TV Shows", marker=dict(color="#a678de"))

    data = [trace1]
    layout = go.Layout(template = 'presentation',margin=dict(l=160, r=80, t=80, b=50),
                       title="Contenu sur "+str(date), font_size= 15, legend=dict(x=0.1, y=1.1, orientation="h"))
    fig = go.Figure(data, layout=layout)
    fig.show()
    return

In [283]:
trace(count_genra(AS_N)[0],count_genra(AS_N)[1])

In [284]:
"""-------------------------Liste des dates où une ou plusieurs séries ont été crées-------------------------"""
liste_date=[]
for i in range(1969,2021):
    if AS_N['Year Released'].isin([i]).sum() != 0:
        liste_date.append(i)
print (liste_date)

[1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]


In [285]:
"""---------------Fonction qui compte les effectifs des genres puis les représente pour une année donnée---------------"""

def count_trace_genra_y (annee,nombre=15):
    
    """On crée le df dont on a besoin à partir de la date voulue"""
    data_f = AS_N[AS_N['Year Released'].isin([annee])]
    
    a=count_genra(data_f,nombre)
    trace(a[0],a[1],nombre,annee)

    return

In [286]:
count_trace_genra_y(2020,10)

In [287]:
"""----------Fonction intermédiaire qui compte les effectifs des genres les plus populaires pour une année donnée----------"""

def prep (annee,nombre=5):
    
    """On récupère les x genres les plus populaires overall """
    counter_list = co.Counter(recup_cat(AS_N)).most_common(nombre)
    counter_list = dict(counter_list)
    print(counter_list)
    
    """On crée le df dont on a besoin à partir de la date voulue"""
    data_f = AS_N[AS_N['Year Released'].isin([annee])]
    
    """On récupère les genres les plus populaires de cette année """    
    categories_y=recup_cat(data_f)
    
    counter_list_y = co.Counter(categories_y).most_common()
    counter_list_y = dict(counter_list_y)
    
    dico=dict()
    for i in counter_list_y.keys():
        if i in counter_list:
            dico[i]=counter_list_y[i]
    
    
    """On conserve le premier élément de chaque couple de counter_list (ie. sur le genre) et on inverse l'ordre de la liste"""
    labels = [key for key in dico.keys()]
    #labels = [_[0] for _ in dico][::-1]
    #print (labels)

    """Idem pour le nombre d'occurrences"""
    values = [value for value in dico.values()]
    #values = [_[1] for _ in dico][::-1]
    return (values,labels)

In [288]:
"""----------Fonction qui compare les effectifs des genres populaires pour 3 années données----------"""

def genra_comp(year1,year2,year3,nombre=5):
    fig = go.Figure(data=[
        go.Bar(name=str(year1), y=prep(year1,nombre)[0], x=prep(year1,nombre)[1]),
        go.Bar(name=str(year2), y=prep(year2,nombre)[0], x=prep(year2,nombre)[1]),
        go.Bar(name=str(year3), y=prep(year3,nombre)[0], x=prep(year3,nombre)[1])
    ])
    # Change the bar mode
    fig.update_layout(barmode='group')
    
    fig.update_layout(template = 'presentation',
                      title='Contenu populaire en '+str(year1)+", "+str(year2)+" et "+str(year3),
                      font_size= 15, legend=dict(yanchor="top", y=0.99, xanchor="right", x=1))
    fig.show()
    return

In [289]:
genra_comp(2015,2017,2020,5)

{'Drama': 575, 'Comedy': 356, 'Documentary': 216, 'Action & Adventure': 211, 'Crime': 206}
{'Drama': 575, 'Comedy': 356, 'Documentary': 216, 'Action & Adventure': 211, 'Crime': 206}
{'Drama': 575, 'Comedy': 356, 'Documentary': 216, 'Action & Adventure': 211, 'Crime': 206}
{'Drama': 575, 'Comedy': 356, 'Documentary': 216, 'Action & Adventure': 211, 'Crime': 206}
{'Drama': 575, 'Comedy': 356, 'Documentary': 216, 'Action & Adventure': 211, 'Crime': 206}
{'Drama': 575, 'Comedy': 356, 'Documentary': 216, 'Action & Adventure': 211, 'Crime': 206}


# Comparaison entre Plateformes

Est-ce que Netflix a plus de séries que les autres?

In [290]:
AS_N= pd.read_csv(r'C:\Users\User\Documents\GitHub\Projet_Python_2A\AS_N.csv',sep=',')
del AS_N['Unnamed: 0']

"""----------On crée les 3 df correspondant aux 3 principaux concurrents de Netflix : Prime Video, Hulu et fuboTV----------"""

AS_P=AS[AS['Streaming Platform'].isin(['Prime Video'])]
AS_H=AS[AS['Streaming Platform'].isin(['Hulu'])]
AS_F=AS[AS['Streaming Platform'].isin(['fuboTV'])]

"""----------On se limite aux années post 1990----------"""

start_year=1990
years=[]
for i in range (start_year,2025):
    years.append(i)
    
AS_N=AS_N[AS_N['Year Released'].isin(years)]
AS_P=AS_P[AS_P['Year Released'].isin(years)]
AS_H=AS_H[AS_H['Year Released'].isin(years)]
AS_F=AS_F[AS_F['Year Released'].isin(years)]

"""----------Graphiques de l'ancienneté des séries sur les Plateformes----------"""

fig = plt.subplots.make_subplots(specs=[[{"secondary_y": True}]])
colors =['#DF0101','#086A87','#58FA82','yellow']


for e,i in enumerate([AS_N,AS_P,AS_H,AS_F]):
    
    cumsumMoviesReleased_i=i['Year Released'].value_counts().sort_index().cumsum()
    trace = go.Scatter(x = list(cumsumMoviesReleased_i.index), y = list(cumsumMoviesReleased_i.values),
                       marker_color=colors[e],
                       mode='lines', name = ['Netflix','Prime','Hulu','fuboTV'][e])
    fig.add_trace(trace, secondary_y=False)


fig.update_layout(template = 'presentation',margin=dict(l=80, r=80, t=25, b=50),
                  title = { 'text' : '<b>Ancienneté du contenu sur les plateformes</b>', 'x' : 0.5, 'y' : 0.94},
                 font_family = 'Calibri',font_size = 16, title_font_color= 'black', showlegend = True)

fig.update_layout(legend=dict(
    yanchor="top",
    y=0.99,
    xanchor="left",
    x=0.01))

fig.show()

FileNotFoundError: [Errno 2] File C:\Users\User\Documents\GitHub\Projet_Python_2A\AS_N.csv does not exist: 'C:\\Users\\User\\Documents\\GitHub\\Projet_Python_2A\\AS_N.csv'

Hypothèse : Prime au dessus de Netflix avant 2019 parce que possède beaucoup de contenu racheté et donc plus ancien que Netflix qui a beaucoup de contenu original crée dans les dernières années d'ou la croissance quasi exponentielle.

Est-ce que Netflix a le meme type de Séries que les autres ?

In [ ]:
"""----------Fonction qui compare les genres les plus populaires en fonction des plateformes----------"""
def comp_platforms_genra(nombre):
    genre_count={}
    genre_count["Netflix"]=count_genra(AS_N,nombre)
    genre_count["Prime"]=count_genra(AS_P,nombre)
    genre_count["Hulu"]=count_genra(AS_H,nombre)
    genre_count["fuboTV"]=count_genra(AS_F,nombre)
    print(genre_count)
    colors =['#DF0101','#086A87','#58FA82','yellow']

    fig = plt.subplots.make_subplots(rows = 2, cols = 2, subplot_titles = ['Netflix','Prime','Hulu','fuboTV'],
                                    horizontal_spacing=0.15,
                                    vertical_spacing=0.2)
    for e,i in enumerate(['Netflix','Prime','Hulu','fuboTV']):
        trace = go.Bar(y = genre_count[i][1], x=genre_count[i][0],
                       marker_color=colors[e],name ="titres" , orientation = 'h')
        fig.add_trace(trace, row = (e//2) + 1, col = (e%2) + 1)

    fig.update_layout(showlegend = False)
    fig.update_layout(template = 'presentation',margin=dict(l=180, r=80, t=70, b=50),
                      title = { 'text' : '<b>Contenu le plus populaire selon les plateformes</b>', 'x' : 0.5},
                     font_family = 'Calibri', font_size = 16, title_font_color= 'black', showlegend = False)
    fig.show()
    return


In [ ]:
comp_platforms_genra(5)

### Genres les mieux notés

In [ ]:
genres = ", ".join(AS_N['Genre']).split(", ")
genres = ",".join(AS_N['Genre']).split(",")
for i in ['2018','2017','2016','2019','2020','Netflix','2015','2012','2007','2004',
              '2010','2013','2009','1969','2005','2006','2014','1999','1972','2002','2011',
              '2008']:
    while i in genres:
        genres.remove(i)
        

In [ ]:
selected_genres=[]
for i in genres :
    if i not in selected_genres:
        selected_genres.append(i)
selected_genres=sorted(selected_genres)

In [ ]:
selected_genres = ['Action & Adventure', 'Animation','Anime','Biography','Children','Comedy','Crime','Cult','DIY', 'Documentary',
                      'Drama','Family','Fantasy','Food','Game Show','History','Home & Garden','Horror',
                      'Lifetime','Musical','Mystery','Pet','Reality','Romance','Science','Science-Fiction','Showtime', 'Sport',
                      'Stand-up & Talk','Thriller','Travel']
selected_genres

ImDB et rotten tomates.

ImDB note sur 10 et Rotten Tomatoes sur 100

In [ ]:
#calculating the overall mean imdb rating and the overall rotten tomatoes rating
overall_mean_imdb = AS_N[AS_N['IMDB Rating'] != -1]['IMDB Rating'].mean()
overall_mean_r = AS_N[AS_N['R Rating'] != -1]['R Rating'].mean()
print (overall_mean_imdb,overall_mean_r)